In [10]:
import RPi.GPIO as GPIO
import time
import serial

# Nastavení GPIO režimu
GPIO.setmode(GPIO.BCM)

# Definice pinů pro senzor, diodu a sériovou komunikaci s Arduinem
senzor_pin = 21
dioda_pin = 12
tlacitko_pin = 18  # Připojte tlačítko na pin 18
ser = serial.Serial('/dev/ttyACM0', 9600, timeout=1)  # '/dev/ttyACM0' může být jiný, závisí na připojení Arduina

# Nastavení pinů
GPIO.setup(senzor_pin, GPIO.IN)
GPIO.setup(dioda_pin, GPIO.OUT)
GPIO.setup(tlacitko_pin, GPIO.IN, pull_up_down=GPIO.PUD_UP)

try:
    dioda_zapnuta = False

    while True:
        # Čtení stavu magnetického senzoru
        stav_senzoru = GPIO.input(senzor_pin)

        # Čtení stavu tlačítka
        tlacitko_stisknuto = not GPIO.input(tlacitko_pin)

        # Zapnutí diody, pouze pokud je senzor sepnutý a tlačítko není stisknuto
        if stav_senzoru == GPIO.HIGH and not dioda_zapnuta and not tlacitko_stisknuto:
            GPIO.output(dioda_pin, GPIO.HIGH)
            dioda_zapnuta = True
            print("Senzor sepnutý - dioda svítí")

        # Vypnutí diody, pouze pokud tlačítko na Arduinu je stisknuto
        if dioda_zapnuta and tlacitko_stisknuto:
            GPIO.output(dioda_pin, GPIO.LOW)
            dioda_zapnuta = False
            print("Dioda vypnutá po stisknutí tlačítka na Arduinu")

        # Kontrola připojení senzoru jednou za dvě sekundy
        time.sleep(2)

        # Posílání informace o stisknutí tlačítka Raspberry Pi
        ser.write(str(int(tlacitko_stisknuto)).encode())
        time.sleep(0.1)

except KeyboardInterrupt:
    # Uzavření GPIO při přerušení
    GPIO.cleanup()
    ser.close()
    print("Program ukončen.")


Senzor sepnutý - dioda svítí
Program ukončen.
